In [1]:
import sys,os
if  not os.path.abspath('./') in sys.path:
    sys.path.append(os.path.abspath('./'))
if  not os.path.abspath('../') in sys.path:
    sys.path.append(os.path.abspath('../'))

import pandas as pd
import numpy as np
import traceback

from dashgrid import dash_df_components as ddfc
dgc = ddfc.dgc
html = dgc.html

### Generate some example data for the Chained Dropdowns

In [2]:
'''
create the Dash app and launch it on args.host and args.port
'''

CONTRACTS_TO_DISPLAY_DICT = {'names':['E-Mini SP','Nymex Crude','Ice Brent','Nymex NatGas'], 
                             'symbols':['ES','CL','CB','NG']
}                             




c1 = CONTRACTS_TO_DISPLAY_DICT['symbols']
c2 = [
    list(range(2014,2021)),list(range(2011,2021)),
    list(range(2016,2021)),list(range(2012,2021))
]
all_months = ['F','G','H','J','K','M','N','Q','U','V','X','Z']
q_months = ['H','M','U','Z']
c3 = [q_months,all_months,all_months,all_months]
c4 = list(zip(c1,c2,c3))
c5 = [(x[0],y,z) for x in c4[:] for y in x[1] for z in x[2]]
contracts = [x[0] for x in c5]
years = [x[1] for x in c5]
months = [x[2] for x in c5]

df_contracts = pd.DataFrame({'commod':contracts,'year':years,'month':months})
df_contracts

,commod,year,month
0,ES,2014,H
1,ES,2014,M
2,ES,2014,U
3,ES,2014,Z
4,ES,2015,H
...,...,...,...
311,NG,2020,Q
312,NG,2020,U
313,NG,2020,V
314,NG,2020,X


In [3]:
logger = dgc.init_root_logger('logfile.log','INFO')

In [4]:
            
mdd = dgc.MultiDropdownDiv('mdd',df_contracts.copy(),['commod','year','month'])            

In [5]:
ac = [mdd]
gc = ['1fr'] 
app = dgc.make_app(ac,grid_template_columns_list=gc)    
app.run_server(host='127.0.0.1',port=8500)    


 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


2020-01-26 20:22:30,445 - werkzeug - INFO -  * Running on http://127.0.0.1:8500/ (Press CTRL+C to quit)
2020-01-26 20:22:36,460 - werkzeug - INFO - 127.0.0.1 - - [26/Jan/2020 20:22:36] "GET / HTTP/1.1" 200 -
2020-01-26 20:22:36,525 - werkzeug - INFO - 127.0.0.1 - - [26/Jan/2020 20:22:36] "GET /_dash-component-suites/dash_renderer/react@16.8.6.min.js?v=1.0.0&m=1574289295 HTTP/1.1" 200 -
2020-01-26 20:22:36,529 - werkzeug - INFO - 127.0.0.1 - - [26/Jan/2020 20:22:36] "GET /_dash-component-suites/dash_renderer/prop-types@15.7.2.min.js?v=1.0.0&m=1574289295 HTTP/1.1" 200 -
2020-01-26 20:22:36,539 - werkzeug - INFO - 127.0.0.1 - - [26/Jan/2020 20:22:36] "GET /_dash-component-suites/dash_renderer/react-dom@16.8.6.min.js?v=1.0.0&m=1574289295 HTTP/1.1" 200 -
2020-01-26 20:22:36,548 - werkzeug - INFO - 127.0.0.1 - - [26/Jan/2020 20:22:36] "GET /_dash-component-suites/dash_html_components/dash_html_components.min.js?v=1.0.0&m=1574289295 HTTP/1.1" 200 -
2020-01-26 20:22:36,550 - werkzeug - INFO - 

In [6]:
DEFAULT_CONFIGS = {"PATH_DATA_HOME":"./",
                  "host":"127.0.0.1",
                  "port":8550,
                  "url_base_pathname":"futskew"}

# read configuration
import json
try:
    configs = json.load(open('./temp_folder/dgrid_components_futures_skew_example_app.json','r'))
    logger.info(f'using configs located at ./temp_folder/dgrid_components_futures_skew_example_app.json')
except:
    traceback.print_exc()
    logger.info(f'using default configs')
    configs = DEFAULT_CONFIGS.copy()

PATH_DATA_HOME = configs['PATH_DATA_HOME']#'../../barchartacs/barchartacs/temp_folder'
FILENAME_SKEW = 'df_iv_skew_COMMOD.csv'
FILENAME_IV = 'df_iv_final_COMMOD.csv'
FILENAME_FUT = 'df_cash_futures_COMMOD.csv'

df_iv_skew = None
df_iv_final = None
df_cash_futures = None
for commod in ['CL','CB','ES','NG']:
    fn_skew = FILENAME_SKEW.replace('COMMOD',commod)
    df_skew = pd.read_csv(f'{PATH_DATA_HOME}/{fn_skew}')
    fn_iv = FILENAME_IV.replace('COMMOD',commod)
    df_iv = pd.read_csv(f'{PATH_DATA_HOME}/{fn_iv}')
    fn_fut = FILENAME_FUT.replace('COMMOD',commod)
    df_fut = pd.read_csv(f'{PATH_DATA_HOME}/{fn_fut}')
    df_skew['commod'] = commod
    df_iv['commod'] = commod
    df_fut['commod'] = commod
    if df_iv_skew is None:
        df_iv_skew = df_skew.copy()
        df_iv_final = df_iv.copy()
        df_cash_futures = df_fut.copy()
    else:
        df_iv_skew = df_iv_skew.append(df_skew.copy())
        df_iv_final = df_iv_final.append(df_iv.copy())
        df_cash_futures = df_cash_futures.append(df_fut.copy())
df_iv_skew = df_iv_skew.rename(columns={c:float(c) for c in df_iv_skew.columns.values if '0.' in c})


2020-01-27 02:46:17,352 - root - INFO - using configs located at ./temp_folder/dgrid_components_futures_skew_example_app.json
/Users/bperlman1/Virtualenvs3/dashrisk3/lib/python3.6/site-packages/pandas/core/frame.py:7138: FutureWarning:

Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.




In [13]:
def plot_skew_vs_atm(SYMBOL_TO_RESEARCH,df_iv_final_in,df_iv_skew_in,df_cash_futures_in,dist_from_zero=.1,year=None):
    # Step 00: select only SYMBOL_TO_RESEARCH from DataFrames 
    df_iv_final = df_iv_final_in[df_iv_final_in.symbol.str.slice(0,2)==SYMBOL_TO_RESEARCH].copy()
    df_iv_skew = df_iv_skew_in[df_iv_skew_in.symbol.str.slice(0,2)==SYMBOL_TO_RESEARCH].copy()
    df_cash_futures = df_cash_futures_in[df_cash_futures_in.symbol.str.slice(0,2)==SYMBOL_TO_RESEARCH].copy()

    year = 'all' if year is None else year
    if str(year).lower() != 'all':
        y = int(str(year))
        beg_year = y*100*100+1*100+1
        end_year = y*100*100+12*100+31
        df_iv_final = df_iv_final[(df_iv_final.settle_date>=beg_year) & (df_iv_final.settle_date<=end_year)]
        df_iv_skew = df_iv_skew[(df_iv_skew.settle_date>=beg_year) & (df_iv_skew.settle_date<=end_year)]
        df_cash_futures = df_cash_futures[(df_cash_futures.settle_date>=beg_year) & (df_cash_futures.settle_date<=end_year)]

    print(f'plot_skew_vs_atm year: {year}')

    # Step 01: create df_skew_2, which holds skew difference between 
    #   positive dist_from_zero and negative dist_from_zero, for each settle_date
    df_skew_2 = df_iv_skew.copy()
    df_skew_2.index.name = None
    skew_range_col = f'iv_skew'
    df_skew_2[skew_range_col] = df_skew_2[dist_from_zero] - df_skew_2[-dist_from_zero]
    df_skew_2.settle_date = df_skew_2.settle_date.astype(int)
    df_skew_2 = df_skew_2[['settle_date',skew_range_col]]
    
    # Step 02: create atm implied vol table, that also has the cash price for each settle_date
    df_atmv = df_iv_final[['settle_date','atm_iv']].drop_duplicates()
    df_cf = df_cash_futures[df_cash_futures.symbol==f'{SYMBOL_TO_RESEARCH}Z99']
    df_atmv = df_atmv.merge(df_cf[['settle_date','close']],how='inner',on='settle_date')
    
    # Step 03: merge skew and atm vol/close tables
    df_ivs = df_skew_2.merge(df_atmv,how='inner',on='settle_date')
    df_ivs = df_ivs.sort_values('settle_date')
    
    # Step 04: plot skew vs atm_iv
    chart_title = f'{SYMBOL_TO_RESEARCH} skew {dist_from_zero*100}% up and down vs atm vol'
    df_ivs_skew_vs_atm_iv = df_ivs[['settle_date',skew_range_col,'atm_iv']]
    fig_skew_vs_atm_iv = plotly_plot(df_ivs_skew_vs_atm_iv,x_column='settle_date',yaxis2_cols=['atm_iv'],
                      y_left_label='iv_skew',y_right_label='atm_iv',plot_title=chart_title)
    
    # Step 05: plot skew vs close
    chart_title = f'{SYMBOL_TO_RESEARCH} skew {dist_from_zero*100}% up and down vs close'
    df_ivs_skew_vs_close = df_ivs[['settle_date',skew_range_col,'close']]
    fig_skew_vs_close = plotly_plot(df_ivs_skew_vs_close,x_column='settle_date',yaxis2_cols=['close'],
                      y_left_label='iv_skew',y_right_label='close',plot_title=chart_title)
    return fig_skew_vs_atm_iv,fig_skew_vs_close
    
def plot_atm_vs_close(SYMBOL_TO_RESEARCH,df_iv_final_in,df_cash_futures_in,year=None):
    # Step 00: select only SYMBOL_TO_RESEARCH from DataFrames 
    df_iv_final = df_iv_final_in[df_iv_final_in.symbol.str.slice(0,2)==SYMBOL_TO_RESEARCH].copy()
    df_cash_futures = df_cash_futures_in[df_cash_futures_in.symbol.str.slice(0,2)==SYMBOL_TO_RESEARCH].copy()
    
    year = 'all' if year is None else year
    if str(year).lower() != 'all':
        y = int(str(year))
        beg_year = y*100*100+1*100+1
        end_year = y*100*100+12*100+31
        df_iv_final = df_iv_final[(df_iv_final.settle_date>=beg_year) & (df_iv_final.settle_date<=end_year)]
        df_cash_futures = df_cash_futures[(df_cash_futures.settle_date>=beg_year) & (df_cash_futures.settle_date<=end_year)]

    print(f'plot_atm_vs_close year: {year}')
    
    # Step 01: create atm implied vol table, that also has the cash price for each settle_date
    df_atmv = df_iv_final[['settle_date','atm_iv']].drop_duplicates()
    df_cf = df_cash_futures[df_cash_futures.symbol==f'{SYMBOL_TO_RESEARCH}Z99']
    df_atmv = df_atmv.merge(df_cf[['settle_date','close']],how='inner',on='settle_date')

    # Step 02: plot atm_iv vs close
    chart_title = f'{SYMBOL_TO_RESEARCH} atm vol vs close'
    df_atm_vs_close = df_atmv[['settle_date','atm_iv','close']]
    fig_atm_vs_close = plotly_plot(df_atm_vs_close,x_column='settle_date',yaxis2_cols=['close'],
                      y_left_label='atm_iv',y_right_label='close',plot_title=chart_title)
    return fig_atm_vs_close

# iplot(plot_atm_vs_close('CB',df_iv_final,df_cash_futures,year=2018))
# for d in [.05,.1,.2]:
#     fig1,fig2 = plot_skew_vs_atm('CB',df_iv_final,df_iv_skew,df_cash_futures,dist_from_zero=d,year=2018)
#     iplot(fig1)
#     iplot(fig2)

### Build callback that returns plotly graph Figures

In [15]:
# define a storage components
#  build a store here
SKEW_RANGE_LIST = [.05,.1,.2]
def create_figs_closure(df_iv_final_in, df_iv_skew_in,df_cash_futures_in):
    def _create_figs(input_list):
        print(f'create_input_store_dict {input_list}')
        if input_list is None or len(input_list)<2 or (input_list[0] is None and input_list[1] is None):
            return {}
        sym_to_plot = input_list[0]
        year_to_plot = 'all' if input_list[1] is None else input_list[1]
        all_figs = []
        fig = plot_atm_vs_close(sym_to_plot,df_iv_final_in,df_cash_futures_in,year=year_to_plot)
        all_figs.append([fig])
        for d in SKEW_RANGE_LIST:
            fig_skew_vs_atm,fig_skew_vs_price = plot_skew_vs_atm(sym_to_plot,df_iv_final_in,df_iv_skew_in,
                                         df_cash_futures_in,dist_from_zero=d,year=year_to_plot)
            all_figs.append([fig_skew_vs_atm])
            all_figs.append([fig_skew_vs_price])
        return all_figs
    return _create_figs


In [8]:
tm_text1 = '''
# Commodity Option Skew Analysis
## Select a Commodity, Year and Monthcode below to display charts showing:
'''
tm1 = ddfc.MarkdownLine(tm_text1,text_size=1)
tm_text2 = '''
* atm vol vs price
* skew vs price
* skew vs atm vol
             '''
tm2 = ddfc.MarkdownLine(tm_text2,text_align='left')
title_comp = ddfc.MarkdownDiv('mdiv1',[tm1,tm2])


In [11]:
dfcf = df_cash_futures[df_cash_futures.commod != 'NG'][['settle_date','commod']]
dfcf['year'] = dfcf.settle_date.apply(lambda v: int(str(v)[0:4]))
dfcf = dfcf[['commod','year']]
mdd2 = dgc.MultiDropdownDiv('mdd2',dfcf.copy(),['commod','year']) 
                                                 

In [20]:
import importlib
importlib.reload(ddfc)

figure_example = ddfc.FigureStatic('figure_example',None,initial_data=None)
my_callback = create_figs_closure(df_iv_final, df_iv_skew,df_cash_futures)
v = ddfc.VariableRowDiv('my_app',
            mdd2.dropdown_list,
            my_callback,
            [figure_example],
            '1fr',
            logger=logger)


2020-01-27 03:52:26,069 - root - INFO - StoreComponent self.output_data_tuple ('store_comp_my_app', 'data')


In [22]:
ac = [title_comp,mdd2] + v.final_components
gc = ['1fr','1fr'] + v.final_layout 
app = dgc.make_app(ac,grid_template_columns_list=gc)    
app.run_server(host='127.0.0.1',port=8500)    


 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


2020-01-27 03:55:03,491 - werkzeug - INFO -  * Running on http://127.0.0.1:8500/ (Press CTRL+C to quit)
2020-01-27 03:55:07,392 - werkzeug - INFO - 127.0.0.1 - - [27/Jan/2020 03:55:07] "GET / HTTP/1.1" 200 -
2020-01-27 03:55:07,443 - werkzeug - INFO - 127.0.0.1 - - [27/Jan/2020 03:55:07] "GET /_dash-component-suites/dash_renderer/react@16.8.6.min.js?v=1.0.0&m=1574289295 HTTP/1.1" 200 -
2020-01-27 03:55:07,460 - werkzeug - INFO - 127.0.0.1 - - [27/Jan/2020 03:55:07] "GET /_dash-component-suites/dash_renderer/react-dom@16.8.6.min.js?v=1.0.0&m=1574289295 HTTP/1.1" 200 -
2020-01-27 03:55:07,468 - werkzeug - INFO - 127.0.0.1 - - [27/Jan/2020 03:55:07] "GET /_dash-component-suites/dash_renderer/prop-types@15.7.2.min.js?v=1.0.0&m=1574289295 HTTP/1.1" 200 -
2020-01-27 03:55:07,476 - werkzeug - INFO - 127.0.0.1 - - [27/Jan/2020 03:55:07] "GET /_dash-component-suites/dash_html_components/dash_html_components.min.js?v=1.0.0&m=1574289295 HTTP/1.1" 200 -
2020-01-27 03:55:07,481 - werkzeug - INFO - 

create_input_store_dict {'mdd2_0': '{}'}
create_input_store_dict {'mdd2_0': 'CL', 'mdd2_1': '2010'}


2020-01-27 03:55:08,357 - werkzeug - INFO - 127.0.0.1 - - [27/Jan/2020 03:55:08] "POST /_dash-update-component HTTP/1.1" 200 -
2020-01-27 03:55:08,411 - root - WARNING - store_transform unexpected conv_key: <class 'dashgrid.dgrid_components.ChainedDropDownDiv'>
2020-01-27 03:55:08,412 - root - WARNING - store_transform unexpected conv_key: <class 'dashgrid.dgrid_components.ChainedDropDownDiv'>
2020-01-27 03:55:08,413 - flask.app - ERROR - Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "/Users/bperlman1/Virtualenvs3/dashrisk3/lib/python3.6/site-packages/flask/app.py", line 2292, in wsgi_app
    response = self.full_dispatch_request()
  File "/Users/bperlman1/Virtualenvs3/dashrisk3/lib/python3.6/site-packages/flask/app.py", line 1815, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/Users/bperlman1/Virtualenvs3/dashrisk3/lib/python3.6/site-packages/flask/app.py", line 1718, in handle_user_exception
    reraise(exc_type, exc_v

create_input_store_dict {'mdd2_0': 'CL', 'mdd2_1': '2010'}
